In [ ]:
import hpmcm
import tables_io
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATADIR = "shear_tract"
shear_st = "0p02"
tract = 10463
SOURCE_TABLEFILES = sorted(glob.glob(os.path.join(DATADIR, f"shear_wmom_{shear_st}_uncleaned_{tract}_*.pq")))
SOURCE_TABLEFILES.reverse()
VISIT_IDS = np.arange(len(SOURCE_TABLEFILES))

REF_DIR = (37.91729, 6.94389)  # RA, DEC in deg
PIXEL_R2CUT = 4

In [ ]:
matcher = hpmcm.Match.createCoaddCellsForTract(pixelR2Cut=PIXEL_R2CUT, pixelMatchScale=1)

In [ ]:
matcher.reduceData(SOURCE_TABLEFILES, VISIT_IDS)

In [ ]:
matcher._nCell

In [ ]:
do_partial = True
if do_partial:
    xRange = range(50, 70)
    yRange = range(170, 190)
    matcher.analysisLoop(xRange, yRange)
else:
    matcher.analysisLoop()

In [ ]:
stats = matcher.extractStats()
shear_stats = matcher.extractShearStats()
obj_shear = shear_stats[1]

In [ ]:
def write_outputs(stats, shear_stats, tract):
    out_dict = dict(
        cluster_assoc=stats[0],
        object_assoc=stats[1],
        cluster_stats=stats[2],
        object_stats=stats[3],
        clsuter_shear=shear_stats[0],
        object_shear=shear_stats[1],
    )
    tables_io.write(out_dict, f"shear_wmom_{shear_st}_match_{tract}_.pq")


In [ ]:
write_outputs(stats, shear_stats, tract)

In [ ]:
mask_0 = stats[0].cat == 0

In [ ]:
_ = plt.hist(matcher.fullData[0].xCell_coadd, bins=np.linspace(-100, 100, 201))
_ = plt.hist(matcher.fullData[0].loc[stats[0][mask_0].id].xCell_coadd, bins=np.linspace(-100, 100, 201))

In [ ]:
_ = plt.hist(matcher.fullData[0].yCell_coadd, bins=np.linspace(-100, 100, 201))
_ = plt.hist(matcher.fullData[0].loc[stats[0][mask_0].id].yCell_coadd, bins=np.linspace(-100, 100, 201))

In [ ]:
objLists = matcher.classifyObjects()
hpmcm.Match.printObjectTypes(objLists)

In [ ]:
n_good = len(objLists['ideal'])
bad_list = ['edge_mixed', 'edge_missing', 'edge_extra', 'orphan', 'missing', 'two_missing', 'many_missing', 'extra', 'caught']
n_bad = np.sum([len(objLists[x]) for x in bad_list])

In [ ]:
n_good/(n_good+n_bad)

In [ ]:
clusterLists = matcher.classifyClusters()
hpmcm.Match.printClusterTypes(clusterLists)

In [ ]:
_ = hpmcm.viz_utils.showShearObjs(matcher, clusterLists['edge_missing'][0])

In [ ]:
_ = hpmcm.viz_utils.showShearObj(matcher, objLists['two_missing'][0])

In [ ]:
_ = hpmcm.viz_utils.showShearObj(matcher, objLists['many_missing'][0])

In [ ]:
_ = hpmcm.viz_utils.showShearObj(matcher, objLists['oprhan'][0])

In [ ]:
_ = hpmcm.viz_utils.showShearObj(matcher, objLists['extra'][0])